<a href="https://colab.research.google.com/github/HelouiseDayane/Cienciasdedados/blob/main/ETL_de_uma_coluna_em_multiplas_colunas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from google.colab import files

df = pd.read_excel('/content/rcl_incor_saep.xls')

# Função para processar e extrair campos
def process_text_to_dict(text):
    if pd.isnull(text):
        return {}

    pattern = r'@#(\d+)@(\d+)&([^@#]*)'
    matches = re.findall(pattern, text)

    # Cria chaves únicas tipo '1385_45'
    result = {}
    for id1, id2, value in matches:
        key = f'{id1}_{id2}'
        result[key] = value.strip()

    return result

# Aplica a função
df_dicts = df['rcl_txt'].apply(process_text_to_dict)

# Transforma lista de dicionários em colunas
df_expandido = pd.DataFrame(df_dicts.tolist())

# Preenche campos nulos com zero
df_expandido = df_expandido.fillna(0)

# Junta com as colunas originais (exceto 'rcl_txt', se quiser)
df_result = pd.concat([df.drop(columns=['rcl_txt']), df_expandido], axis=1)

# Exibe resultado
df_result.head()
# Exportar para Excel
df_result.to_excel('/content/resultado_convertido.xlsx', index=False, engine='openpyxl')
files.download('/content/resultado_convertido.xlsx')



In [ ]:
import pandas as pd
from google.colab import files

# Garante que todas as colunas sejam strings
df_result.columns = df_result.columns.astype(str)

# Função para gerar e salvar o arquivo baseado nas colunas
def gerar_excel(df, nome_arquivo, col_inicio, col_fim):
    if col_inicio in df.columns and col_fim in df.columns:
        idx_inicio = df.columns.get_loc(col_inicio)
        idx_fim = df.columns.get_loc(col_fim)

        # Seleciona as colunas do intervalo
        colunas_intervalo = df.columns[idx_inicio:idx_fim + 1]
        # Colunas fixas
        colunas_fixas = ['rcl_dthr', 'rcl_pac']

        # Filtra linhas com pelo menos um valor diferente de zero no intervalo
        filtro = ~((df[colunas_intervalo] == 0).all(axis=1))

        # Aplica o filtro e seleciona colunas fixas + intervalo
        df_filtrado = df.loc[filtro, colunas_fixas + list(colunas_intervalo)]

        # Salva
        caminho = f'/content/{nome_arquivo}.xlsx'
        df_filtrado.to_excel(caminho, index=False, engine='openpyxl')
        files.download(caminho)
    else:
        print(f'❌ Colunas {col_inicio} ou {col_fim} não encontradas.')

# === Arquivo 1 ===
# rcl_dthr, rcl_pac, 1385_45 até 2360_61
gerar_excel(df_result, 'arquivo_1_1385a2360', '1385_45', '2360_61')

# === Arquivo 2 ===
# rcl_dthr, rcl_pac, 1423_8 até 1596_67
gerar_excel(df_result, 'arquivo_2_1423a1596', '1423_8', '1596_67')

# === Arquivo 3 ===
# rcl_dthr, rcl_pac, 1377_92 até 2360_91
gerar_excel(df_result, 'arquivo_3_1377a2360', '1377_92', '2360_91')
